In [1]:
import pandas as pd
import os
path="../../Data/"
import numpy as np
import re
import datetime as dt
disease_cols_df=pd.read_csv(path+'cols_disease.csv')


In [2]:
def findcols(df,string):
    return [col for col in df if string in col]

In [3]:
dis_cols=list(disease_cols_df['column'])

In [4]:
%%time
ukb_tp0_dis=pd.read_parquet(path+'ukb_tp0_new.parquet',columns=dis_cols)

CPU times: user 18.3 s, sys: 4.39 s, total: 22.7 s
Wall time: 12.8 s


In [5]:
for col in ukb_tp0_dis.columns:
    ukb_tp0_dis[col][(ukb_tp0_dis[col]=='nan')]=np.NaN

<ipython-input-5-753cbe0487df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukb_tp0_dis[col][(ukb_tp0_dis[col]=='nan')]=np.NaN


In [6]:
%%time
dis_full=pd.DataFrame([])
for i,col in enumerate(ukb_tp0_dis.columns):
    if 'eid' not in col and 'assessment_centre' not in col:
        print(i)
        df1=ukb_tp0_dis[[col,'eid','date_of_attending_assessment_centre_f53_0_0']][pd.notnull(ukb_tp0_dis[col])]
        df1.columns=['disease_date','eid','date_assess']
        df1['disease_date']=pd.to_datetime(df1['disease_date']).dt.date
        df1['date_assess']=pd.to_datetime(df1['date_assess']).dt.date
        df1['disease']=str(col)
        dis_full=pd.concat([dis_full,df1],axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
dis_full['disease']=dis_full['disease'].str.replace('date_|_first_|reported_','',regex=True)

In [8]:
mask_bef=(dis_full['disease_date']<dis_full['date_assess'])

mask_aft=(dis_full['disease_date']>=dis_full['date_assess']+pd.offsets.DateOffset(years=2))&\
(dis_full['disease_date']<=dis_full['date_assess']+pd.offsets.DateOffset(years=10))

mask_10y=(dis_full['disease_date']>dis_full['date_assess']+pd.offsets.DateOffset(years=10))

dis_full['dis_bef']=0
dis_full['dis_bef'][mask_bef]=1

dis_full['dis_aft']=0
dis_full['dis_aft'][mask_aft]=1

dis_full['dis_exc']=0
dis_full['dis_exc'][mask_10y|mask_bef]=1

<ipython-input-8-8b51c90da82c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_full['dis_bef'][mask_bef]=1
<ipython-input-8-8b51c90da82c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_full['dis_aft'][mask_aft]=1
<ipython-input-8-8b51c90da82c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_full['dis_exc'][mask_10y|mask_bef]=1


In [9]:
dis_full['total_bef']=dis_full.groupby('disease')['dis_bef'].transform('sum')
dis_full['total_aft']=dis_full.groupby('disease')['dis_aft'].transform('sum')

In [10]:
dis_full[(dis_full['total_aft']>200)].shape[0]/dis_full.shape[0]

0.9593145228084554

In [11]:
dis_sum=pd.DataFrame(dis_full.groupby('disease')['dis_bef','dis_aft'].sum()).reset_index()

<ipython-input-11-9b1c5a4f795c>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dis_sum=pd.DataFrame(dis_full.groupby('disease')['dis_bef','dis_aft'].sum()).reset_index()


In [12]:
%%time
dis_ohe=pd.DataFrame(dis_full.groupby(['eid','disease'])['dis_bef'].sum().unstack('disease')).reset_index()

CPU times: user 1min 44s, sys: 10min 13s, total: 11min 58s
Wall time: 18min 39s


In [19]:
'''
bring back null records
'''
dis_ohe=pd.merge(ukb_tp0_dis['eid'],dis_ohe,how='left',on='eid')
dis_ohe.fillna(0,inplace=True)

In [20]:
dis_ohe['total_dis']=dis_ohe[[col for col in dis_ohe.columns if 'eid' not in col]].sum(axis=1)

In [21]:
diseases_words_monitor='dementia|alzheim|parkin|hunting|diabetes'
mask_dis=(dis_full['disease'].str.contains(diseases_words_monitor,regex=True))

In [22]:
labels=pd.DataFrame(dis_full[mask_dis].groupby(['eid','disease'])['dis_aft'].sum().unstack('disease')).reset_index()
labels.columns=['eid']+[col+'_label' for col in labels.columns if col!='eid']
labels=pd.merge(ukb_tp0_dis['eid'],labels,how='left',on='eid')
labels.fillna(0,inplace=True)

In [23]:
excludes=pd.DataFrame(dis_full[mask_dis].groupby(['eid','disease'])['dis_exc'].sum().unstack('disease')).reset_index()
excludes=pd.merge(ukb_tp0_dis['eid'],excludes,how='left',on='eid')
excludes.fillna(0,inplace=True)
excludes.columns=['eid']+[col+'_exc' for col in labels.columns if col!='eid']

In [24]:
dis_ohe.to_parquet(path+'dis_ohe.parquet')
labels.to_parquet(path+'labels.parquet')
excludes.to_parquet(path+'excludes.parquet')

In [43]:
findcols(dis_sum,'dementia')

[]

In [101]:
dis_ohe[dis_ohe[[col for col in dis_ohe.columns if 'parkinsons' in col]].sum(axis=1)>0]['eid'].count()

2723

In [45]:
dis_sum[dis_sum['disease'].str.contains('dementia')]

,disease,dis_bef
263,f00dementia_in_alzheimers_disease_f130836_0_0,1956
264,f01vascular_dementia_f130838_0_0,1555
265,f02dementia_in_other_diseases_classified_elsew...,809
266,f03unspecified_dementia_f130842_0_0,3585


In [46]:
dis_sum[dis_sum['disease'].str.contains('park')]

,disease,dis_bef
355,g20parkinsons_disease_f131022_0_0,2723
356,g21secondary_parkinsonism_f131024_0_0,220
357,g22parkinsonism_in_diseases_classified_elsewhe...,4
